In [8]:
from datasets import load_dataset
ds = load_dataset(path="ai4bharat/IndicCorpV2", split= "hin_Deva", streaming="True")

In [9]:
ds2 = load_dataset(path="ai4bharat/IndicCorpV2", split= "guj_Gujr", streaming="True")

In [ ]:
from itertools import islice

for i, example in enumerate(islice(ds2, 5)):
    print(f"Example {i+1}:")
    print(example)
    print("------")


Example 1:
{'text': 'આ વીડિયો જુઓ: ઊંઝા માર્કેટયાર્ડ આજથી 25 જુલાઈ સુધી બંધ'}
------
Example 2:
{'text': ''}
------
Example 3:
{'text': 'મિથેનોલ આવ્યો ક્યાંથી?'}
------
Example 4:
{'text': ''}
------
Example 5:
{'text': 'આખરે ત્રણ રાજ્યોમાં મળેલ હાર પર કોંગ્રેસ અધ્યક્ષ રાહુલ ગાંધી દ્વારા પ્રથમ પ્રતિક્રિયા આપવામાં આવી છે. તેમણે કહ્યું કે, ત્રિપુરા, નાગાલેન્ડ અને મેઘાલયમાં લોકોના જનાદેશનો સ્વાગત કરીએ છે અને આ ક્ષેત્રના લોકોનો વિશ્વાસ ફરીથી જીતીવા માટે પ્રતિબદ્ધ છીએ.'}
------


In [ ]:
for i, sample in enumerate(ds2):
    print(sample)
    if i == 2:  
        break

{'text': 'આ વીડિયો જુઓ: ઊંઝા માર્કેટયાર્ડ આજથી 25 જુલાઈ સુધી બંધ'}
{'text': ''}
{'text': 'મિથેનોલ આવ્યો ક્યાંથી?'}


In [ ]:
#para to sentence tokenizer 

import re

def gujarati_sentence_tokenizer(text):
    # Replace multiple newlines or whitespace with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Add a marker to split on Gujarati and English sentence-ending punctuations
    sentence_endings = r'[।!?]|(?<!\d)\.(?!\d)'  # Avoid splitting on decimal points like 3.14
    sentences = re.split(sentence_endings, text)
    
    # Strip leading/trailing whitespace and remove empty sentences
    sentences = [s.strip() for s in sentences if s.strip()]
    
    return sentences

In [11]:
gujarati_paragraph = "મારું નામ રવિ છે. હું અમદાવાદમાં રહું છું! શું તું મને ઓળખે છે? હું બહુ ખુશ છું।"
sentences = gujarati_sentence_tokenizer(gujarati_paragraph)

for i, sentence in enumerate(sentences, 1):
    print(f"Sentence {i}: {sentence}")


Sentence 1: મારું નામ રવિ છે
Sentence 2: હું અમદાવાદમાં રહું છું
Sentence 3: શું તું મને ઓળખે છે
Sentence 4: હું બહુ ખુશ છું


In [12]:
texts = []
for i, sample in enumerate(ds2):
    if "text" in sample:
        texts.append(sample["text"])
    if i >= 999:
        break

print("Example text:", texts[0])


Example text: આ વીડિયો જુઓ: ઊંઝા માર્કેટયાર્ડ આજથી 25 જુલાઈ સુધી બંધ


In [ ]:
#sentence to words 

def gujarati_word_tokenizer(text):
    text = re.sub(r'[“”"\'‘’]', '', text)   #  quotes
    text = re.sub(r'[\n\r\t]', ' ', text)   #  newlines/tabs with space
    text = re.sub(r'[।|]', '.', text)       # danda or pipes to period
    text = re.sub(r'\s+', ' ', text)        # collapse multiple spaces
    text = re.sub(r'([.,!?()])', r' \1 ', text)  # isolate punctuation
    text = re.sub(r'\s{2,}', ' ', text)        # clean again

    #Extracting guj words
    gujarati_words = re.findall(r'[\u0A80-\u0AFF]+', text)
    
    #  Extracting punctuation
    punctuation_tokens = re.findall(r'[.,!?()]', text)

    all_tokens = re.findall(r'[\u0A80-\u0AFF]+|[.,!?()]', text)

    return all_tokens

sentence = "હું market જઇ રહ્યો છું because I want to buy fruits અને દૂધ!"
tokens = gujarati_word_tokenizer(sentence)

print("Tokens:", tokens)
    


Tokens: ['હું', 'જઇ', 'રહ્યો', 'છું', 'અને', 'દૂધ', '!']


In [ ]:
#         r'\d+\.\d+|' #floating point
#         r'\d+|' #integer
#         r'[\w\.-]+@[\w\.-]+|' #email-address
#         r'\w+://\S+|' #URL
#         r'[^\s\w]', #punctuation and special characters

In [ ]:
output_file = "tokenized_output.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for i, example in enumerate(ds2):
        paragraph = example["text"]
        
        sentences = gujarati_sentence_tokenizer(paragraph)
        
        for sentence in sentences:
            tokens = gujarati_word_tokenizer(sentence)
            if tokens:
                f.write(" ".join(tokens) + "\n")

        if i == 999:
            break

print(f"Tokenized sentences saved to {output_file}")

Tokenized sentences saved to tokenized_output.txt


In [ ]:
import json

all_sentences = []
all_tokens = []

for i, example in enumerate(ds2):
    paragraph = example["text"]
    
    sentences = gujarati_sentence_tokenizer(paragraph)
    all_sentences.extend(sentences)

    for sentence in sentences:
        tokens = gujarati_word_tokenizer(sentence)
        clean_tokens = [tok for tok in tokens if tok.strip() not in ['', '।', '|', '.', ',']]  # filter noise
        if clean_tokens:
            all_tokens.append(clean_tokens)

    if i == 999:
        break

output_data = {
    "sentences": all_sentences,
    "tokens": all_tokens
}

output_file = "tokenized.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

print(f"✅ Output saved to {output_file}")


✅ Output saved to tokenized.json


In [ ]:
import json

with open("tokenized.json", "r", encoding="utf-8") as f:
    data = json.load(f)

tokens_per_sentence = data.get("tokens", [])

total_sentences = len(tokens_per_sentence)

words = [word for sentence in tokens_per_sentence for word in sentence]
total_words = len(words)

total_characters = sum(len(word) for word in words)

avg_sentence_length = total_words / total_sentences if total_sentences > 0 else 0

avg_word_length = total_characters / total_words if total_words > 0 else 0

unique_tokens = set(words)
ttr = len(unique_tokens) / total_words if total_words > 0 else 0

print(f"Total Sentences: {total_sentences}")
print(f"Total Words: {total_words}")
print(f"Total Characters: {total_characters}")
print(f"Average Sentence Length: {avg_sentence_length:.2f}")
print(f"Average Word Length: {avg_word_length:.2f}")
print(f"Type/Token Ratio (TTR): {ttr:.4f}")


Total Sentences: 1548
Total Words: 21856
Total Characters: 106508
Average Sentence Length: 14.12
Average Word Length: 4.87
Type/Token Ratio (TTR): 0.3781
